In [ ]:
#DATASETS:  https://www.kaggle.com/competitions/daasbstp2023/data

In [1]:
#Let's start by importing all packages needed.
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import seaborn as sns
from sklearn import preprocessing
import numpy as np
import time
from dateutil import parser
import pytz

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree, export_text


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

from scipy.stats import uniform, poisson

from sklearn.metrics import confusion_matrix, \
                            accuracy_score, \
                            precision_score, \
                            recall_score, \
                            f1_score, \
                            fbeta_score, \
                            mean_squared_error, \
                            mean_absolute_error, \
                            roc_auc_score, \
                            roc_curve, \
                            classification_report


# **Sample and Assess**

In [2]:
#Load the datasets.
df_en1 = pd.read_csv('daasbstp2023/energia_202109-202112.csv', na_filter=False, encoding = "latin")
df_en2 = pd.read_csv('daasbstp2023/energia_202201-202212.csv', na_filter=False, encoding = "latin")
df_me1 = pd.read_csv('daasbstp2023/meteo_202109-202112.csv', na_filter=False, encoding = "latin")
df_me2 = pd.read_csv('daasbstp2023/meteo_202201-202212.csv', na_filter=False, encoding = "latin")

In [3]:
#Append the datasets by type.
df_en = df_en1._append(df_en2,ignore_index=True)
#df_en = df_en._append(df_en3,ignore_index=True)  #test X

df_me = df_me1._append(df_me2,ignore_index=True)
#df_me = df_me._append(df_me3,ignore_index=True)  #test X

## Energy Consumption Dataset

This dataset provides information on energy consumption recorded at different timestamps. Each record includes the following columns:

- **Data (Date):** The timestamp associated with the record, indicating the date.
- **Hora (Hour):** The hour associated with the record.
- **Normal (kWh):** The amount of electrical energy consumed, in kilowatt-hours (kWh), from the electrical grid during a normal period in daily bi-hourly cycles (non-off-peak hours).
- **Horário Económico (kWh):** The amount of electrical energy consumed, in kilowatt-hours (kWh), from the electrical grid during an economic period in daily bi-hourly cycles (off-peak hours).
- **Autoconsumo (kWh):** The amount of electrical energy consumed, in kilowatt-hours (kWh), generated from solar panels (self-consumption).
- **Injeção na rede (kWh):** A qualitative feature indicating the level of energy injection into the grid on a scale of None, Low, Medium, High, and Very High.

This dataset is utilized in a competition and includes various features, with particular emphasis on the "Injeção na rede (kWh)" feature, which provides information about the qualitative scale of energy injection into the grid.

In [4]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016 entries, 0 to 11015
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Data                     11016 non-null  object 
 1   Hora                     11016 non-null  int64  
 2   Normal (kWh)             11016 non-null  float64
 3   Horário Económico (kWh)  11016 non-null  float64
 4   Autoconsumo (kWh)        11016 non-null  float64
 5   Injeção na rede (kWh)    11016 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 516.5+ KB


In [5]:
df_en.iloc[901].to_frame().T

,Data,Hora,Normal (kWh),Horário Económico (kWh),Autoconsumo (kWh),Injeção na rede (kWh)
901,2021-11-05,13,0.0,0.0,0.274,Very High


In [6]:
df_en.isna().any()

Data                       False
Hora                       False
Normal (kWh)               False
Horário Económico (kWh)    False
Autoconsumo (kWh)          False
Injeção na rede (kWh)      False
dtype: bool

In [7]:
# Unique values for each column in the Energy dataset.
for column in df_en.columns:
    unique_values = df_en[column].unique()
    print(f"{column}, Number of Unique Values: {len(unique_values)}")

Data, Number of Unique Values: 459
Hora, Number of Unique Values: 24
Normal (kWh), Number of Unique Values: 1282
Horário Económico (kWh), Number of Unique Values: 851
Autoconsumo (kWh), Number of Unique Values: 752
Injeção na rede (kWh), Number of Unique Values: 5


## Weather Information Dataset

This dataset provides comprehensive information related to weather conditions at different timestamps. The columns in the dataset include:

- **dt (Timestamp):** The timestamp associated with the record.
- **dt_iso (ISO Date):** The date associated with the record, accurate to the second.
- **city_name:** The location for which weather data is recorded.
- **temp (Temperature):** The temperature in degrees Celsius.
- **feels_like (Feels Like):** The perceived temperature in degrees Celsius.
- **temp_min (Min Temperature):** The minimum felt temperature in degrees Celsius.
- **temp_max (Max Temperature):** The maximum felt temperature in degrees Celsius.
- **pressure:** Atmospheric pressure in atmospheres.
- **sea_level:** Atmospheric pressure at sea level in atmospheres.
- **grnd_level:** Atmospheric pressure at local altitude in atmospheres.
- **humidity:** Humidity level as a percentage.
- **wind_speed:** Wind speed in meters per second.
- **rain_1h (Rainfall):** Average precipitation value in the last hour.
- **clouds_all (Cloudiness):** Cloud coverage percentage.
- **weather_description:** Qualitative assessment of weather conditions.


In [8]:
df_me.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11688 entries, 0 to 11687
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dt                   11688 non-null  int64  
 1   dt_iso               11688 non-null  object 
 2   city_name            11688 non-null  object 
 3   temp                 11688 non-null  float64
 4   feels_like           11688 non-null  float64
 5   temp_min             11688 non-null  float64
 6   temp_max             11688 non-null  float64
 7   pressure             11688 non-null  int64  
 8   sea_level            11688 non-null  object 
 9   grnd_level           11688 non-null  object 
 10  humidity             11688 non-null  int64  
 11  wind_speed           11688 non-null  float64
 12  rain_1h              11688 non-null  object 
 13  clouds_all           11688 non-null  int64  
 14  weather_description  11688 non-null  object 
dtypes: float64(5), int64(4), object(6)
m

In [9]:
df_me.iloc[801].to_frame().T

,dt,dt_iso,city_name,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,rain_1h,clouds_all,weather_description
801,1633338000,2021-10-04 09:00:00 +0000 UTC,local,14.03,13.84,13.34,14.54,1023,,,90,2.07,,69,broken clouds


In [10]:
df_me.isna().any()

dt                     False
dt_iso                 False
city_name              False
temp                   False
feels_like             False
temp_min               False
temp_max               False
pressure               False
sea_level              False
grnd_level             False
humidity               False
wind_speed             False
rain_1h                False
clouds_all             False
weather_description    False
dtype: bool

In [11]:
# Unique values for each column in the Energy dataset.
for column in df_me.columns:
    unique_values = df_me[column].unique()
    print(f"{column}, Number of Unique Values: {len(unique_values)}")

dt, Number of Unique Values: 11688
dt_iso, Number of Unique Values: 11688
city_name, Number of Unique Values: 1
temp, Number of Unique Values: 2423
feels_like, Number of Unique Values: 2702
temp_min, Number of Unique Values: 488
temp_max, Number of Unique Values: 550
pressure, Number of Unique Values: 41
sea_level, Number of Unique Values: 1
grnd_level, Number of Unique Values: 1
humidity, Number of Unique Values: 82
wind_speed, Number of Unique Values: 771
rain_1h, Number of Unique Values: 372
clouds_all, Number of Unique Values: 101
weather_description, Number of Unique Values: 8


# **Data Processing**

After Loading the data, we must now process it in order to be able to use it in our model. For this we will follow the following steps:
1. Remove unnecessary columns
2. Handle the Date column on both datasets and unify the format for later joining of the datasets
3. Handle the missing values or rows (dates that are not present in both datasets)
4. Handle the categorical values

In [12]:
# We can drop city_name, sea_level and grnd_level as they only have one unique value
df_me = df_me.drop(['city_name', 'sea_level', 'grnd_level'], axis=1)

In [13]:
# Convert columns to categorical
df_en["Injeção na rede (kWh)"] = df_en["Injeção na rede (kWh)"].astype("category")
df_me["weather_description"] = df_me['weather_description'].astype("category")

# For "Injeção na rede (kWh)" column in df_en
print("Categories for 'Injeção na rede (kWh)' in df_en:")
print(df_en["Injeção na rede (kWh)"].cat.categories)
print("\n")

# For "weather_description" column in df_me
print("Categories for 'weather_description' in df_me:")
print(df_me["weather_description"].cat.categories)
print("\n")


Categories for 'Injeção na rede (kWh)' in df_en:
Index(['High', 'Low', 'Medium', 'None', 'Very High'], dtype='object')


Categories for 'weather_description' in df_me:
Index(['broken clouds', 'few clouds', 'heavy intensity rain', 'light rain',
       'moderate rain', 'overcast clouds', 'scattered clouds', 'sky is clear'],
      dtype='object')




In [14]:
# Convert columns to unified format
df_en['datetime'] = pd.to_datetime(df_en['Data'] + ' ' + df_en['Hora'].astype(str) + ':00:00', format='%Y-%m-%d %H:%M:%S')

# Drop the original 'Data' and 'Hora' columns if needed
df_en = df_en.drop(['Data', 'Hora'], axis=1)

# Print the updated DataFrame
df_en.iloc[901].to_frame().T

,Normal (kWh),Horário Económico (kWh),Autoconsumo (kWh),Injeção na rede (kWh),datetime
901,0.0,0.0,0.274,Very High,2021-11-05 13:00:00


In [18]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016 entries, 0 to 11015
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Normal (kWh)             11016 non-null  float64       
 1   Horário Económico (kWh)  11016 non-null  float64       
 2   Autoconsumo (kWh)        11016 non-null  float64       
 3   Injeção na rede (kWh)    11016 non-null  category      
 4   datetime                 11016 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), float64(3)
memory usage: 355.3 KB


In [15]:
#fix datetime in df_me (loses the UTC tag)
df_me['dt_iso'] = pd.to_datetime(df_me['dt_iso'], format='%Y-%m-%d %H:%M:%S %z UTC')
df_me['dt_iso'] = df_me['dt_iso'].dt.tz_localize(None)
df_me = df_me.rename(columns={"dt_iso": "datetime"})
df_me.iloc[801].to_frame().T

,dt,datetime,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,rain_1h,clouds_all,weather_description
801,1633338000,2021-10-04 09:00:00,14.03,13.84,13.34,14.54,1023,90,2.07,,69,broken clouds


In [17]:
# We can also drop the 'dt' column as it is redundant
df_me = df_me.drop(['dt'], axis=1)

In [19]:
# Order the dataframes by datetime so we can detect any time skips
df_en = df_en.sort_values(by=['datetime'])
df_me = df_me.sort_values(by=['datetime'])

In [20]:
time_diff_en = df_en['datetime'].diff()
time_diff_me = df_me['datetime'].diff()

# Print the irregular time intervals
irregularities_en = time_diff_en[time_diff_en != '0 days 01:00:00']
irregularities_me = time_diff_me[time_diff_me != '0 days 01:00:00']
print("Irregular time intervals in df_en:")
print(irregularities_en)
print("\n")
print("Irregular time intervals in df_me:")
print(irregularities_me)


Irregular time intervals in df_en:
0   NaT
Name: datetime, dtype: timedelta64[ns]


Irregular time intervals in df_me:
0   NaT
Name: datetime, dtype: timedelta64[ns]


In [21]:
outer_join_merged_df = pd.merge(df_en, df_me, on='datetime', how='outer')

In [23]:
outer_join_merged_df.isna().sum()

Normal (kWh)               672
Horário Económico (kWh)    672
Autoconsumo (kWh)          672
Injeção na rede (kWh)      672
datetime                     0
temp                         0
feels_like                   0
temp_min                     0
temp_max                     0
pressure                     0
humidity                     0
wind_speed                   0
rain_1h                      0
clouds_all                   0
weather_description          0
dtype: int64

Since the datetime data exhibited no irregularities, the additional entries present in the Weather dataset but not in the Energy dataset can be attributed to the Weather dataset containing data from days before or after the Energy dataset's first or last entry, respectively. A manual analysis of the dataset reveals that the Weather dataset includes entries starting from 2021-09-01, while the Energy dataset commences from 2021-09-29. Consequently, it is necessary to exclude entries from the Weather dataset that precede 2021-09-29, as they will not contribute to the modeling process. In order to achieve this we will do an Inner Join between the two datasets on the datetime column, and the resulting dataset will be the one we use going forward.

In [26]:
#at this point, if I do an inner join I am losing 5.7% of all datarows
merged_df = pd.merge(df_en, df_me, on='datetime', how='inner')

In [28]:
#Sava the clean dataset
merged_df.to_csv('merged.csv')

In [30]:
merged_df.iloc[901].to_frame().T

,Normal (kWh),Horário Económico (kWh),Autoconsumo (kWh),Injeção na rede (kWh),datetime,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,rain_1h,clouds_all,weather_description
901,0.0,0.0,0.274,Very High,2021-11-05 13:00:00,15.75,14.92,13.12,17.88,1027,59,3.06,,1,sky is clear
